<a href="https://colab.research.google.com/github/t4toast/project-one/blob/main/Sands_project_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project one, final
- Diane T Sands

-steps to remember:
1. import all libraries
2. Load the dataset
3. make copy of df
5. check about dataset info
6. df.isna().sum().sum()
7. check for duplicates and drop the duplicates - df.duplicated().sum()
8. check for incosistacneis
9. Split the dataset-train/test
10. column selection - separate the categorical and numerical one data
11. handling missing values using imputer for categorical and numerical dataset
12. justify the strategy used in code like why are you using mean, median or mode.
13. Make pipleines
14. use column transformers for scaling , ohe, for the categorical and numerical data.
15.fit the data to train and transform train/test both


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
from sklearn import set_config
set_config(display='diagram')

In [31]:
#load the dataset
filename = '/content/drive/MyDrive/Colab Notebooks/sales_predictions.csv'
df = pd.read_csv(filename)

In [32]:
#make a copy of the data
ml_df = df.copy()

In [33]:
#explore the data some 
ml_df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [34]:
#check for missing data
ml_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [35]:
#check for duplicates
ml_df.duplicated().sum()

0

In [36]:
#no duplicated rows. Can drop columns Item Identifier & Outlet Year 
#Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target 
#and the rest of the relevant variables as your features matrix.  
X = ml_df.drop(['Item_Identifier', 'Outlet_Establishment_Year', 'Item_Outlet_Sales'], axis=1, inplace=True)
y = ml_df['Item_Outlet_Sales']

KeyError: ignored

In [ ]:
#Perform a train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.head()

In [ ]:
#Make sure your imputation of missing values occurs  after the train test split using SimpleImputer.  
#Item Weight and Outlet Size are the two categories with missing data.
#look again at the data...

ml_df['Item_Weight'].describe()  

In [ ]:
ml_df['Outlet_Size'].value_counts()

In [ ]:
#Create a ColumnTransformer to preprocess the data. 
cat_selector = make_column_selector(dtype_include= 'object')
num_selector = make_column_selector(dtype_include= 'number')
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown= 'drop')
mean_imputer = SimpleImputer(strategy='mean')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')
#using mean for Item Weight as it is very close to 50% of the data

#make tuples
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')
col_transformer.fit(X_train)
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

In [ ]:
#make pipelines
number_pipe = make_pipeline(mean_imputer, scaler)
number_pipe


In [ ]:
categorical_pipe = make_pipeline(most_frequent, ohe)
categorical_pipe

In [ ]:
#preprocessor
preprocessor = make_column_transformer(categorical_tuple,
                                       number_tuple,
                                       remainder='drop')

In [ ]:
#define the MAE, MSE, RMSE and R2 series of code to use later
def evaluate_regression(y_true, y_pred):
  
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_true, y_pred)

  print(f'scores: \nMAE: {mae:,.2f} \nMSE: {mse:,.2f} \nRMSE: {rmse:,.2f} \nR2: {r2:.2f}')


In [ ]:
#linear regression model
lin_reg = LinearRegression()

lin_reg_pipe = make_pipeline(preprocessor, lin_reg)

lin_reg_pipe.fit(X_train, y_train)

In [ ]:
#run evaluation parameters
print('Training')
evaluate_regression(y_train, dec_tree_pipe.predict(X_train))
print('Testing')
evaluate_regression(y_test, dec_tree_pipe.predict(X_test))

In [ ]:
#do a decision tree
dec_tree = DecisionTreeRegressor(random_state = 42)
dec_tree.fit(X_train, y_train)


In [ ]:
#run evaluation parameters
print('Training')
evaluate_regression(y_train, dec_tree_pipe.predict(X_train))
print('Testing')
evaluate_regression(y_test, dec_tree_pipe.predict(X_test))

I have have tried 2 different models on the data set. I need to determine which model to implement.

Overall, which model do you recommend? >>> neither yet since neither of them worked.
Justify your recommendation.